In [1]:
import os
import requests
import pandas as pd

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

authKey = os.getenv('KMA_API_KEY')

In [3]:
category='typ01/url/kma_sfctm3.php' #ASOS hourly

In [4]:
def create_url(authKey, category, **kwargs):
    url = f'https://apihub.kma.go.kr/api/{category}?'
    for key, value in kwargs.items():
        url += f'{key}={value}&'
    url += f'authKey={authKey}'
    return url

In [5]:
params={
    "tm1": 202308010000,
    "tm2": 202308022359,
    "stn": 0,
    "help": 0,
}

url = create_url(authKey, category, **params)

In [6]:
response = requests.get(url) 
res_txt = response.text

lines = res_txt.strip().split('\n')
print(res_txt[:2000])
print("...")
print("...")
print("...")
print(res_txt[-810:])

data_rows = [line.split() for line in lines]
data = data_rows[4:-2]

#START7777
#2345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234
# YYMMDDHHMI STN  WD   WS GST  GST  GST     PA     PS PT    PR    TA    TD    HM    PV     RN     RN     RN     RN     SD     SD     SD WC WP WW                      CA  CA   CH CT        CT  CT  CT    VS   SS    SI ST    TS    TE    TE    TE    TE  ST   WH BF IR IX
#        KST  ID  16  m/s  WD   WS   TM    hPa    hPa  -   hPa     C     C     %   hPa     mm    DAY    JUN    INT    HR3    DAY    TOT -- -- ---------------------- TOT MID  MIN -------- TOP MID LOW                  GD     C     5    10    20    30 SEA    m --  -  -
202308010000  90  34  1.0  -9 -9.0   -9 1010.0 1012.0  8  -0.1  25.5  22.5  84.0  27.2   -9.0   -9.0   -9.0   -9.0   -9.0   -9.0   -9.0 -9 -9 -                        3   0   -9 -         -9  -9  -9  

In [7]:
lines = res_txt.strip().split('\n')
data_rows = [line.split() for line in lines]
data = data_rows[4:-2]

In [8]:
def create_header(url):
    url = url.replace('help=0','help=1')
    response = requests.get(url) 
    res_txt = response.text
    lines = res_txt.strip().split('\n')
    
    for line_number, line in enumerate(lines, start=1):
        if '01234567890123456789' in line:
            endline = line_number-2
            break
            
    header_rows = [line.split() for line in lines[4:endline]]
    header=[]
    for i in header_rows:
        header.append(i[2])
    return header

header = create_header(url)

In [9]:
# Make Dataframe 
df = pd.DataFrame(data, columns=header)
int_col = ['TM','STN','WD','GST_WD','GST_TM','PT','WC','WP','CA_TOT','CA_MID','CH_MIN','CT_TOP','CT_MID','CT_LOW']
obj_col = ['CT','WW']
flt_col = list(set(header) - set(int_col+obj_col))

for i in int_col:
    df[i] = df[i].astype(int)

for i in flt_col:
    df[i] = df[i].astype(float)


df

,TM,STN,WD,WS,GST_WD,GST_WS,GST_TM,PA,PS,PT,...,TS,TE_005,TE_01,TE_02,TE_03,ST_SEA,WH,BF,IR,IX
0,202308010000,90,34,1.0,-9,-9.0,-9,1010.0,1012.0,8,...,24.5,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
1,202308010000,93,0,0.0,-9,-9.0,-9,1001.8,1012.7,0,...,27.5,29.7,30.5,31.2,31.1,-9.0,-9.0,-9.0,3.0,2.0
2,202308010000,95,9,1.4,-9,-9.0,-9,994.9,1012.5,0,...,26.1,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
3,202308010000,98,11,0.5,-9,-9.0,-9,998.9,1012.0,4,...,28.1,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,2.0
4,202308010000,99,7,0.6,-9,-9.0,-9,1009.2,1012.7,0,...,26.6,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645,202308022300,285,25,0.8,-9,-9.0,-9,1003.9,1006.9,-9,...,29.4,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
4646,202308022300,288,34,0.6,-9,-9.0,-9,1006.0,1006.9,-9,...,28.0,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
4647,202308022300,289,16,1.0,-9,-9.0,-9,992.5,1008.1,-9,...,27.1,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
4648,202308022300,294,23,0.6,-9,-9.0,-9,1002.6,1007.7,-9,...,27.0,-99.0,-99.0,-99.0,-99.0,-9.0,-9.0,-9.0,3.0,-9.0
